In [26]:
from flask import Flask, request, jsonify
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json
import pandas as pd
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok



In [25]:
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=4f784c9d93afd754e0aa83a55c4792956faed95d315a73ed9ea6c356072667ed
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [19]:
!pip uninstall flask-ngrok
!pip install flask-ngrok

Found existing installation: flask-ngrok 0.0.25
Uninstalling flask-ngrok-0.0.25:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/flask_ngrok-0.0.25.dist-info/*
    /usr/local/lib/python3.10/dist-packages/flask_ngrok.py
Proceed (Y/n)? Y
  Successfully uninstalled flask-ngrok-0.0.25
  Using cached flask_ngrok-0.0.25-py3-none-any.whl (3.1 kB)


In [20]:
preprocessed_data = pd.read_csv('/content/drive/MyDrive/Dissertation/preprocessed_data.csv')

In [21]:
original_data = pd.read_csv('/content/drive/MyDrive/Dissertation/synthetic_updated_data.csv')

In [22]:
#Will use this dataset for mapping
original_data['Budget'] = original_data['Transportationcost'] + original_data['Accommodationcost']
original_data = original_data.drop(['Accommodationcost', 'Transportationcost'], axis=1)

In [ ]:
# User's input (replace these values with user's input)
user_gender = 'Male'
user_nationality = 'American'
user_accommodation = 'Hotel'
user_transportation = 'Plane'
user_budget = 1800
#user_duration_of_vacation = 7
#user_age = 35.0

In [ ]:
#Create a DataFrame containing the user's input
user_input = pd.DataFrame({
    'Travelergender': [user_gender],
    'Travelernationality': [user_nationality],
    'Accommodationtype': [user_accommodation],
    'Transportationtype': [user_transportation],
    'Budget': [user_budget],
    #'Duration(days)': [user_duration_of_vacation],
    #'Travelerage': [user_age]
})

In [8]:
original_data.dtypes

Destination             object
Duration(days)           int64
Travelerage            float64
Travelergender          object
Travelernationality     object
Accommodationtype       object
Transportationtype      object
Budget                   int64
dtype: object

In [28]:
app = Flask(__name__)

run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/recommend', methods=['POST'])
def recommend_destinations():
    user_input = request.json  # User input received from Android app
    #user_vector = preprocess_user_input(user_input)  # Preprocess user input
    user_input_df = pd.DataFrame([user_input])
    print(user_input_df)
    #Mapping all the categorical column
    nationality_mapping = {'American': 0, 'Canadian': 7, 'Korean': 23, 'British': 4, 'Vietnamese': 40, 'Australian': 1, 'Brazilian': 3, 'Dutch': 10, 'Emirati': 11, 'Mexican': 24, 'Spanish': 33, 'Chinese': 9, 'German': 13, 'Moroccan': 25, 'Scottish': 27, 'Japanese': 22, 'Italian': 19, 'Indian': 17, 'South Korean': 31, 'French': 12, 'South African': 29, 'Taiwanese': 35, 'Indonesian': 18, 'USA': 37, 'Canada': 6, 'South Korea': 30, 'UK': 36, 'China': 8, 'Taiwan': 34, 'Japan': 21, 'Spain': 32, 'Brazil': 2, 'Germany': 14, 'Hong Kong': 16, 'United Kingdom': 39, 'Singapore': 28, 'Italy': 20, 'Greece': 15, 'United Arab Emirates': 38, 'Cambodia': 5, 'New Zealander': 26}
    user_input_df['Travelernationality'] = user_input_df['Travelernationality'].map(nationality_mapping)

    accommodationType_mapping = {'Airbnb': 0, 'Guesthouse': 1, 'Hostel': 2, 'Hotel': 3, 'Resort': 4, 'Riad': 5, 'Vacation rental': 6, 'Villa': 7}
    user_input_df['Accommodationtype'] = user_input_df['Accommodationtype'].map(accommodationType_mapping)

    transporationType_mapping = {'Bus': 0, 'Car': 1, 'Car rental': 2, 'Ferry': 3, 'Plane': 4, 'Subway': 5, 'Train': 6}
    user_input_df['Transportationtype'] = user_input_df['Transportationtype'].map(transporationType_mapping)

    gender_mapping = {'Female': 0, 'Male': 1}
    user_input_df['Travelergender'] = user_input_df['Travelergender'].map(gender_mapping)
    print(user_input_df)

    #As the original data type of budget column in user input will be object, need to convert it into int or float.
    user_input_df['Budget'] = user_input_df['Budget'].astype(int)

     # Original 'Budget' values in user_input data
    original_budgets = user_input_df['Budget']  # Replace with your actual 'Budget' values

  #Getting min and max value of budget column to calculate normalisation for user input
    print(user_input_df.dtypes)
    min_of_budget = original_data['Budget'].min()
    max_of_budget = original_data['Budget'].max()
  # Calculate min-max scaled values
    value = [(x - min_of_budget) / (max_of_budget - min_of_budget) for x in original_budgets]

    user_input_df['Budget'] = value

    print(user_input_df)

    # Calculate cosine similarity with all destinations
    cosine_scores = cosine_similarity(user_input_df, preprocessed_data.drop(columns=['Destination']))
    print(cosine_scores)
    # Get indices of top recommended destinations
    top_indices = np.argsort(cosine_scores[0])[::-1][:5]

    # Get the corresponding destination names
    recommended_destinations = preprocessed_data['Destination'].iloc[top_indices]

    # Return recommendations in JSON format
    return jsonify({'recommendations': recommended_destinations.tolist()})

if __name__ == '__main__':
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c77f-34-31-24-255.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/Aug/2023 03:48:20] "POST /recommend HTTP/1.1" 200 -


  Travelergender Travelernationality Accommodationtype Transportationtype  \
0           Male            American             Hotel              Plane   

  Budget  
0   1800  
   Travelergender  Travelernationality  Accommodationtype  Transportationtype  \
0               1                    0                  3                   4   

  Budget  
0   1800  
Travelergender         int64
Travelernationality    int64
Accommodationtype      int64
Transportationtype     int64
Budget                 int64
dtype: object
   Travelergender  Travelernationality  Accommodationtype  Transportationtype  \
0               1                    0                  3                   4   

     Budget  
0  0.150508  
[[1.         0.61017451 0.3055519  ... 0.74174602 0.1791602  0.11053552]]


INFO:werkzeug:127.0.0.1 - - [29/Aug/2023 04:53:56] "POST /recommend HTTP/1.1" 200 -


  Travelergender Travelernationality Accommodationtype Transportationtype  \
0           Male            American             Hotel              Plane   

  Budget  
0   1800  
   Travelergender  Travelernationality  Accommodationtype  Transportationtype  \
0               1                    0                  3                   4   

  Budget  
0   1800  
Travelergender         int64
Travelernationality    int64
Accommodationtype      int64
Transportationtype     int64
Budget                 int64
dtype: object
   Travelergender  Travelernationality  Accommodationtype  Transportationtype  \
0               1                    0                  3                   4   

     Budget  
0  0.150508  
[[1.         0.61017451 0.3055519  ... 0.74174602 0.1791602  0.11053552]]


INFO:werkzeug:127.0.0.1 - - [29/Aug/2023 04:54:53] "POST /recommend HTTP/1.1" 200 -


  Travelergender Travelernationality Accommodationtype Transportationtype  \
0           Male            American             Hotel              Plane   

  Budget  
0   1800  
   Travelergender  Travelernationality  Accommodationtype  Transportationtype  \
0               1                    0                  3                   4   

  Budget  
0   1800  
Travelergender         int64
Travelernationality    int64
Accommodationtype      int64
Transportationtype     int64
Budget                 int64
dtype: object
   Travelergender  Travelernationality  Accommodationtype  Transportationtype  \
0               1                    0                  3                   4   

     Budget  
0  0.150508  
[[1.         0.61017451 0.3055519  ... 0.74174602 0.1791602  0.11053552]]


In [27]:
!ngrok authtoken 2U7lYbiZQbpVQ3nYYkwlzzu6Eqx_395SQMWDe8R8YqfVcK4Pw

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [23]:
!ngrok config add-authtoken 2U7lYbiZQbpVQ3nYYkwlzzu6Eqx_395SQMWDe8R8YqfVcK4Pw

/bin/bash: line 1: ngrok: command not found
